In [2]:
import pandas as pd 
import glob
import xarray as xr
import time
import datetime as dt

In [35]:
path = "data/48"

more_files = sorted(glob.glob(path + "/*.dat"))[:20] 
#prøverunnde med noen filer. denne finner alle filene som slutterpå ".dat" og leser bare de 10 første
#ikke utvit til noe særlig mer en typ 20-30 stasjoner. da sprenger pcen. burde gå bedre når vi effektiviserer etterpå


first = True
start = time.time()
for i in more_files:
    
    df = pd.read_table(i,skiprows=2, delim_whitespace=True,skipfooter=1,engine="python")
    
    
    if(df.values.shape[0]<4): # hopper over de korteste profilene
        continue
    
    meta = pd.read_table(i,skiprows=None,sep="\s+",nrows=1,engine="python")
    print(df.values.shape[0], int(str(meta.head().columns[3])[:-1]))
    
    measurement_time = pd.to_datetime(float(meta.values[0,1]),origin=str(int(meta.values[0,0])),unit="D")
    measurement_lat  = float(meta.values[0,2])
    measurement_lon  = float(meta.values[0,3])
    #print(measurement_time)
    
    
    if "%year" in df.columns:
        df["%year"] = df["%year"].astype(int)
        df["times"] = pd.to_datetime(df["day"], unit = 'D', 
                                     origin = str(df["%year"][0]))
        df = df.drop(["%year","day"],axis=1)
    if "nobs" in df.columns:
        df = df.drop("nobs",axis=1)
    if "nacm" in df.columns:
        df = df.drop("nacm",axis=1)
    
    
    df.rename(columns={"%pressure(dbar)":"sea_water_pressure",
                       "pressure(dbar)":"sea_water_pressure",
                       "temperature(C)":"sea_water_temperature",
                       "salinity":"sea_water_salinity",
                       "dissolved_oxygen":"moles_of_oxygen_per_unit_mass_in_sea_water",
                       "oxygen(umol/kg)":"moles_of_oxygen_per_unit_mass_in_sea_water",
                       "CDOM(ppb)":"concentration_of_colored_dissolved_organic_matter_in_sea_water_expressed_as_equivalent_mass_fraction_of_quinine_sulfate_dihydrate",
                       "turbidity(/m/sr)x10^4":"sea_water_turbidity",
                       "chlorophyll-a(ug/l)":"mass_concentration_of_chlorophyll_a_in_sea_water",
                       "PAR(uE/m^2/s)":"downwelling_photosynthetic_radiative_flux_in_sea_water",
                       "east(cm/s)":"eastward_sea_water_velocity",
                       "north(cm/s)":"northward_sea_water_velocity",
                       "vert(cm/s)":"upward_sea_water_velocity"
                       }, inplace=True)
    df = df.set_index("sea_water_pressure")

    ds = xr.Dataset.from_dataframe(df)
    
    ds["time"] = measurement_time
    ds["latitude"]  = measurement_lat
    ds["longitude"]  = measurement_lon
    
    #profile_nr = int(list(pd.read_table(i,sep="[:, ]",nrows=0,engine="python"))[4])
    profile_nr = int(str(meta.head().columns[3])[:-1])
    
    
    ds = ds.assign_coords(profile=profile_nr)
    ds = ds.expand_dims("profile")

    
    # joining files
    if first==True:
        buoy= ds
        first=False
    else:
        buoy=xr.concat([buoy,ds],dim = "profile")
        #concat gjør at det blir mye nan i temp og salinity, men det følger cf. hør med Steingod

print("Det tok", time.time()-start)


#La oss fikse litt metadata da
#legg til alle de her greiene: https://adc.met.no/node/4
#lager metadata:
units = {"time":"seconds since 1970-01-01 00:00:00+0",
         "latitude":"degree_north",
         "longitude":"degree_east",
         "sea_water_pressure":"dBar",
         "sea_water_salinity":"1e-3",
         "sea_water_temperature":"celsius",
         "sea_water_turbidity":"m-1 sr-1 x 10e-4",
         "moles_of_oxygen_per_unit_mass_in_sea_water":"umol/kg",
         "mass_concentration_of_chlorophyll_a_in_sea_water":"ug/l",
         "downwelling_photosynthetic_radiative_flux_in_sea_water":"uE/m^2/s",
         "concentration_of_colored_dissolved_organic_matter_in_sea_water_expressed_as_equivalent_mass_fraction_of_quinine_sulfate_dihydrate":"ppb",
         "eastward_sea_water_velocity":"cm/s",
         "northward_sea_water_velocity":"cm/s",
         "upward_sea_water_velocity":"cm/s"
        }

for i in buoy:
    if i == "times": #behandler times for seg selv da dette er tidspunkt for individuelle målinger 
        continue
    buoy[i].attrs["standard_name"] = i
    buoy[i].attrs["units"] = units[i]

#global attributes
buoy.attrs["title"] = "Nan"
buoy.attrs["summary"] = "Nan"
buoy.attrs["keywords"] = "Nan"
buoy.attrs["keywords_vocabulary"] = "Nan"
buoy.attrs["featureType"] = "trajectoryProfile"

buoy.attrs["geospatial_lat_min"] = min(buoy.latitude.values)
buoy.attrs["geospatial_lat_max"] = max(buoy.latitude.values)
buoy.attrs["geospatial_lon_min"] = min(buoy.longitude.values)
buoy.attrs["geospatial_lon_max"] = max(buoy.longitude.values)

buoy.attrs["time_coverage_start"] = min(buoy.time.values)
buoy.attrs["time_coverage_end"] = max(buoy.time.values)

buoy.attrs["Conventions"] = "ACDD-1.3"
buoy.attrs["history"] = "Nan"
buoy.attrs["date_created"] = "Nan"
buoy.attrs["creator_type"] = "Nan"
buoy.attrs["creator_institution"] = "Nan"
buoy.attrs["creator_name"] = "Nan"
buoy.attrs["creator_email"] = "Nan"
buoy.attrs["creator_url"] = "Nan"
buoy.attrs["project"] = "Nan"
buoy.attrs["license"] = "Nan"



    
for i in buoy:
    print(buoy[i].attrs)

print(dt.datetime(1970,1,1))

780 1
195 2
210 3
144 4
195 5
748 6
742 7
192 8
193 9
196 10
196 11
757 12
759 13
196 14
197 15
198 16
197 17
756 18
757 19
198 20
Det tok 0.5436904430389404
{'standard_name': 'sea_water_temperature', 'units': 'celsius'}
{'standard_name': 'sea_water_salinity', 'units': '1e-3'}
{'standard_name': 'moles_of_oxygen_per_unit_mass_in_sea_water', 'units': 'umol/kg'}
{'standard_name': 'concentration_of_colored_dissolved_organic_matter_in_sea_water_expressed_as_equivalent_mass_fraction_of_quinine_sulfate_dihydrate', 'units': 'ppb'}
{'standard_name': 'downwelling_photosynthetic_radiative_flux_in_sea_water', 'units': 'uE/m^2/s'}
{'standard_name': 'sea_water_turbidity', 'units': 'm-1 sr-1 x 10e-4'}
{'standard_name': 'mass_concentration_of_chlorophyll_a_in_sea_water', 'units': 'ug/l'}
{'standard_name': 'time', 'units': 'seconds since 1970-01-01 00:00:00+0'}
{'standard_name': 'latitude', 'units': 'degree_north'}
{'standard_name': 'longitude', 'units': 'degree_east'}
1970-01-01 00:00:00


In [ ]:
def itp_ascii_to_NETcdf(path):
    

Info:
====
**infoen er forløpig basert på gjennomgang av bare en bøye + at jeg har sett litt på hva som står om de andre. Så den kan være ugenerell, oppdaterer fortløpende. **


Data fra bøyene inholder profiler med trykk, temp og salt. 
Hver profil har en .dat fil med en header som inneholder starttid for profilen, lat,long og hvor mange målinger fila inneholder. enhet for tid er ÅR DAG, der dag har desimaler som gir klokkeslett. dag begynner på 1, ikke null. 

Noen bøyer har også andre målinger: bøya jeg nå ser på, 114, har to fastmonterte microcatsensorer som måler på 5 og 6 meters dyp hvert kvarter. de ligger i filene som har micro i navnet og er mye større. 

Jobber i første omgang med å lage et program som henter data fra én profil og en bøye.


Lage et program som dumper dette til netCDF-CF. 


Tanker og Spørsmål:
============



Ser ut til at metadata og discover-metadata må hardcodes for å stemme med standarene for [metadata]() og [discover-metadata](https://adc.met.no/node/4)

Skal jeg bruke trajectoryProfile eller Profile

skal jeg inkludere mlinger fra f.eks microcat-sensorene 